In [1]:
from icecream import ic
import pandas as pd
from dateutil.relativedelta import relativedelta
# from dhs_preprocessing_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Initialization
pandarallel.initialize()

from dhs_preprocessing_functions import *

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
/home/sven/pycharm/FoodSecurity/water_sven/


In [2]:
# folder: path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
education_folder = f"/mnt/datadisk/data/preprocessed_data/DHS_n_more/"
img_path = '/mnt/datadisk2/preprocessed/all_new/996x996_c432_fillmean_rlocal channel mean_clipvoutlier2.5_normZ_f37363/'

locations_f = f"{input_dir}locations.csv"
sustainbench_f = f"{education_folder}additional_data/sustainbench_w_gaul.csv"
out_f = f"{input_dir}sustainbench_women_bmi2.csv"

value_col = 'women_bmi'
households_col = 'n_women_bmi'

In [3]:
locations_df = pd.read_csv(locations_f)
sustainbench_df = pd.read_csv(sustainbench_f)

/tmp/ipykernel_3170678/2080121114.py:2: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  sustainbench_df = pd.read_csv(sustainbench_f)


In [4]:
locations_df['Sustainbench ID'] = locations_df['DHSCC'] + '-' + locations_df['DHSYEAR'].astype(str) + '-' + \
    locations_df['GEID'].str[4] + '#-' + locations_df['DHSCLUST'].astype(str).str.zfill(8)
locations_df['Sustainbench ID']

0        AO-2006-5#-00000001
1        AO-2006-5#-00000002
2        AO-2006-5#-00000003
3        AO-2006-5#-00000004
4        AO-2006-5#-00000005
                ...         
70751    ZW-2015-7#-00000396
70752    ZW-2015-7#-00000397
70753    ZW-2015-7#-00000398
70754    ZW-2015-7#-00000399
70755    ZW-2015-7#-00000400
Name: Sustainbench ID, Length: 70756, dtype: object

In [5]:
sustainbench_df_final = pd.merge(locations_df, sustainbench_df[['DHSID_EA', 'cname',
       'year', 'lat', 'lon', 'n_asset', 'asset_index', 'n_water',
       'water_index', 'n_sanitation', 'sanitation_index', 'under5_mort',
       'n_under5_mort', 'women_edu', 'women_bmi', 'n_women_edu', 'n_women_bmi']], left_on='Sustainbench ID', right_on='DHSID_EA', how='left')
# sustainbench_df_final[['lat', 'lon', 'LATNUM', 'LONGNUM', 'DHSID_EA', 'Sustainbench ID']]
print(len(sustainbench_df_final))
print(len(sustainbench_df_final['lat'].dropna()))
print(len(sustainbench_df_final['LATNUM'].dropna()))
print(sustainbench_df_final.columns)
sustainbench_df_final['location deviation'] = np.sqrt((sustainbench_df_final['LATNUM'] - sustainbench_df_final['lat'])**2 + (sustainbench_df_final['LONGNUM'] - sustainbench_df_final['lon'])**2)
sustainbench_df_final[sustainbench_df_final['location deviation'] > 0.000001][['lat', 'lon', 'LATNUM', 'LONGNUM', 'DHSID_EA', 'Sustainbench ID', 'location deviation']]
sustainbench_df_final['location deviation'].value_counts()
sustainbench_df_final['households'] = sustainbench_df_final[households_col]


70756
38836
70756
Index(['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS',
       'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS',
       'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM',
       'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name',
       'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code',
       'adm0_name', 'Shape_Area', 'Sustainbench ID', 'DHSID_EA', 'cname',
       'year', 'lat', 'lon', 'n_asset', 'asset_index', 'n_water',
       'water_index', 'n_sanitation', 'sanitation_index', 'under5_mort',
       'n_under5_mort', 'women_edu', 'women_bmi', 'n_women_edu',
       'n_women_bmi'],
      dtype='object')


In [6]:
cluster_df7 = sustainbench_df_final.copy()
cluster_df8 = cluster_df7.copy()
print('1', 'adm0_name' in cluster_df8.columns)
inputs_split_df_accounting_for_mean = []
inputs_split_label_df = []
stats_d = {}
for mode in [
    'split: random', 
    # 'split: random 10splits',
    # 'split: non overlapping',
    'split: out of country',
    # 'split: out of adm1',
    'split: out of adm2',
        ]:
    for sub_df_ind in ['all']:
        for img_p in [img_path]:#, False]:
            for excl_outlier in [False, 'std']:
                # for random in [False, True]:
                for year in [False, True]:
                    # excl_outlier = False
                    # for force_into in [True, False]:
                    for since_year in [2012]:#, 2015]:
                        split_col_n = mode
                        #skip stuff which does not make sense
                        if year:
                            #does not make sense for these
                            if 'split: random' in split_col_n or 'split: random without <2015' in split_col_n:
                                continue
                        # if not random:
                        #     # if 'adm2' in split_col_n:
                        #     #     print('skipping adm2 split_df_accounting_for_mean')
                        #     #     #algo split_df_accounting_for_mean not working for adm2
                        #     #     continue
                        #     if 'split: random' in split_col_n or 'split: random without <2015' in split_col_n:
                        #         continue
                        # if force_into:
                        #     #skip all which
                        #     if not 'split: out of country' in split_col_n:
                        #         continue

                        #define the column which will be used for calculation
                        # if sub_df_ind == 'all':
                        #     value_col = 'PCA w_location_weighting all'
                        # elif sub_df_ind == 'rural':
                        #     value_col = 'PCA w_location_weighting rural'
                        # elif sub_df_ind == 'urban':
                        #     value_col = 'PCA w_location_weighting urban'
                        # else:
                        #     raise NotImplementedError()

                        if year:
                            split_col_n += ' year'
                        split_col_n += ' ' + sub_df_ind
                        # if not img_p:
                        #     split_col_n += ' (wo TIF)'
                        if since_year:
                            split_col_n += f' {since_year}plus'
                        if excl_outlier == 'std':
                            split_col_n += ' excluded outlier surveys'
                        elif excl_outlier is True:
                            split_col_n += ' excluded drop surveys (ZAuEG)'
                        random = False
                        if 'random' in split_col_n:
                            random = True
                        # if random:
                        #     #no doulbe random in name
                        #     if 'split: random' in split_col_n or 'split: random without <2015' in split_col_n:
                        #         pass
                        #     else:
                        #         split_col_n += ' random'
                        splits = 6
                        if '10splits' in mode:
                            splits = 10

                        # else:
                        #     raise ValueError(f'Should not happen: random is {random} and year is {year}')
                        force_test_ind = False
                        force_into = False
                        # if 'split: out of country' in split_col_n:
                        #     force_test_ind = 'Mozambique'
                        #     if force_into:
                        #         split_col_n += ' MZ in test'
                        #     else:
                        #         split_col_n += ' MZ in train'
                        #     if 'year' in split_col_n:
                        #         force_test_ind = ('Mozambique', 2018)

                        force_split_col = False
                        if 'split: out of country' in split_col_n:
                            force_split_col = 'adm0_name'
                        elif 'split: out of adm1' in split_col_n:
                            force_split_col = 'adm1_name'
                        elif 'split: out of adm2' in split_col_n:
                            force_split_col = 'adm2_name'

                        df_in = cluster_df8
                        if 'split: non overlapping' in split_col_n:
                            #calc distance for overlapping, use 10% buffer to ensure with haversine, use pythagoras to ensure there are no overlaps in the corners as well, using 5km radius for now
                            dist = np.sqrt((5 * 1.1) ** 2 + (5 * 1.1) ** 2)
                            df_in = cluster_df8[cluster_df8['DHSYEAR'] >= since_year]
                            df_in = gu.cluster_coordinates(df_in, dist, assign_noise_to_groups=30)
                            numeric_cols = df_in.select_dtypes(include=[np.number]).columns.tolist()
                            for col in ['clustered']:
                                if col in numeric_cols:
                                    numeric_cols.remove(col)
                            clustered = df_in[df_in['clustered'] != -1].groupby('clustered')[numeric_cols].mean()
                            clustered = clustered.reset_index()
                            clustered.to_csv(f'{out_f[:-4]}_clustered_{sub_df_ind}_since{since_year}_{dist:.2f}km.csv', index=False)
                            force_split_col = 'clustered: grouped non-clustered'

                        print(split_col_n)
                        if not random:
                            # print('split_df_accounting_for_mean', split_col_n, force_test_ind, value_col)
                            # print(f'sub_df_ind {sub_df_ind}, value_col {value_col}, force_split_col {force_split_col}, split_col_n {split_col_n}, img_p {img_p}, value_column {excl_outlier}, force_test_ind={force_test_ind}')
                            # print('in', sub_df_ind, value_col, force_split_col, split_col_n, img_p, excl_outlier, force_test_ind, force_into)
                            inputs_split_df_accounting_for_mean.append((df_in.copy(), sub_df_ind, value_col, force_split_col, split_col_n, img_p, 
                                                                        excl_outlier, force_test_ind, force_into, since_year, splits, input_dir))
                            # cluster_df7s = split_df_accounting_for_mean(cluster_df7, sub_df_ind, value_col, force_split_col, split_col_n, img_p, excl_outlier, force_test_ind=force_test_ind, force_into=force_into)
                            # cluster_df7[split_col_n] = cluster_df7s
                        else:
                            # print('split_label_df', split_col_n, force_test_ind, value_col)
                            inputs_split_label_df.append((df_in.copy(), sub_df_ind, value_col, force_split_col, split_col_n, img_p, excl_outlier,
                                                            force_test_ind, force_into, since_year, splits))
                            # cluster_df7 = split_label_df(cluster_df7, value_col, sub_df_ind, split_col_n, force_split_col, force_test_ind=force_test_ind, img_path=img_p, excl_outlier=excl_outlier, force_into=force_into)
                        # visualize_splits(cluster_df7, value_col, split_col_n, force_split_col)
                        # stats_d = stats_of_splits(cluster_df7, value_col, split_col_n, force_split_col, stats_d)

def helper(params, func):
    return func(*params)

#calc split_df_accounting_for_mean
print('starting calc', len(inputs_split_df_accounting_for_mean))
with ProcessPoolExecutor() as executor:
    results1 = executor.map(partial(helper, func=split_df_accounting_for_mean), inputs_split_df_accounting_for_mean)
    results1 = list(results1)
# results1 = []
# for inp in inputs_split_df_accounting_for_mean:
#     results1.append(split_df_accounting_for_mean(*inp))

1 True
split: random all 2012plus
split: random all 2012plus excluded outlier surveys
split: out of country all 2012plus
split: out of country year all 2012plus
split: out of country all 2012plus excluded outlier surveys
split: out of country year all 2012plus excluded outlier surveys
split: out of adm2 all 2012plus
split: out of adm2 year all 2012plus
split: out of adm2 all 2012plus excluded outlier surveys
split: out of adm2 year all 2012plus excluded outlier surveys
starting calc 8


dropping EGGE61FL amount: 1518 mean: 29.84280507535194 vs 23.868426776516934
dropping EGGE61FL amount: 1518 mean: 29.84280507535194 vs 23.868426776516934
dropping EGGE61FL amount: 1518 mean: 29.84280507535194 vs 23.868576929598255
dropping EGGE61FL amount: 1518 mean: 29.84280507535194 vs 23.868576929598255


In [7]:
cluster_df7[['DHSID', 'Sustainbench ID', 'women_bmi']]

,DHSID,Sustainbench ID,women_bmi
0,AO200600000001,AO-2006-5#-00000001,NaN
1,AO200600000002,AO-2006-5#-00000002,NaN
2,AO200600000003,AO-2006-5#-00000003,NaN
3,AO200600000004,AO-2006-5#-00000004,NaN
4,AO200600000005,AO-2006-5#-00000005,NaN
...,...,...,...
70751,ZW201500000396,ZW-2015-7#-00000396,23.163000
70752,ZW201500000397,ZW-2015-7#-00000397,24.330455
70753,ZW201500000398,ZW-2015-7#-00000398,23.648846
70754,ZW201500000399,ZW-2015-7#-00000399,26.576286


In [8]:

for inp, df1 in zip(inputs_split_df_accounting_for_mean, results1):
    # print('inp', inp)
    print('a', list(df1.columns), len(df1))
    print('cdf', list(cluster_df7.columns), len(cluster_df7))
    print(inp[4])
    print(len(cluster_df7), print(len(df1)))
    print(len(cluster_df7['DHSID'].unique()), len(df1['DHSID'].unique()))
    cluster_df7 = pd.merge(cluster_df7, df1[['DHSID', inp[4]]], on='DHSID', how='left')
print(cluster_df7.columns)
print('finished calc split_df_accounting_for_mean')
cluster_df7.to_csv(out_f, index=False)
cluster_df7

a ['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'Shape_Area', 'Sustainbench ID', 'DHSID_EA', 'cname', 'year', 'lat', 'lon', 'n_asset', 'asset_index', 'n_water', 'water_index', 'n_sanitation', 'sanitation_index', 'under5_mort', 'n_under5_mort', 'women_edu', 'women_bmi', 'n_women_edu', 'n_women_bmi', 'location deviation', 'households', 'split: out of country all 2012plus'] 70756
cdf ['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code',

a ['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'Shape_Area', 'Sustainbench ID', 'DHSID_EA', 'cname', 'year', 'lat', 'lon', 'n_asset', 'asset_index', 'n_water', 'water_index', 'n_sanitation', 'sanitation_index', 'under5_mort', 'n_under5_mort', 'women_edu', 'women_bmi', 'n_women_edu', 'n_women_bmi', 'location deviation', 'households', 'split: out of adm2 all 2012plus excluded outlier surveys'] 70756
cdf ['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'ad

,country,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,...,location deviation,households,split: out of country all 2012plus,split: out of country year all 2012plus,split: out of country all 2012plus excluded outlier surveys,split: out of country year all 2012plus excluded outlier surveys,split: out of adm2 all 2012plus,split: out of adm2 year all 2012plus,split: out of adm2 all 2012plus excluded outlier surveys,split: out of adm2 year all 2012plus excluded outlier surveys
0,Angola,AO200600000001,AO,2006,1,AO,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,AO200600000002,AO,2006,2,AO,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AO200600000003,AO,2006,3,AO,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Angola,AO200600000004,AO,2006,4,AO,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AO200600000005,AO,2006,5,AO,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70751,Zimbabwe,ZW201500000396,ZW,2015,396,ZW,NaN,NaN,NaN,NaN,...,0.0,20.0,split 5,split 1,split 3,split 3,split 5,split 5,split 4,split 4
70752,Zimbabwe,ZW201500000397,ZW,2015,397,ZW,NaN,NaN,NaN,NaN,...,0.0,22.0,split 5,split 1,split 3,split 3,split 1,split 0,split 1,split 5
70753,Zimbabwe,ZW201500000398,ZW,2015,398,ZW,NaN,NaN,NaN,NaN,...,0.0,26.0,split 5,split 1,split 3,split 3,split 0,split 0,split 3,split 1
70754,Zimbabwe,ZW201500000399,ZW,2015,399,ZW,NaN,NaN,NaN,NaN,...,0.0,35.0,split 5,split 1,split 3,split 3,split 5,split 5,split 4,split 4


In [9]:
# print(list(cluster_df7.columns))
for d in inputs_split_df_accounting_for_mean:
    visualize_splits(cluster_df7, d[2], d[4], input_dir)
    stats_d = stats_of_splits(cluster_df7, d[2], d[4], d[3], stats_d)
print('starting calc', len(inputs_split_label_df))
#calc split_df
# inputs_split_label_df = inputs_split_label_df[:10]
# results2 = []
# for inp in inputs_split_label_df:
#     results2.append(split_label_df(*inp))
with ProcessPoolExecutor() as executor:
    results2 = executor.map(partial(helper, func=split_label_df), inputs_split_label_df)
# concat
for inp, df1 in zip(inputs_split_label_df, results2):
#     print('inp', inp)
#     print('cdf', list(cluster_df7.columns))
#     print('in df', list(df.columns))
    cluster_df7 = pd.merge(cluster_df7, df1[['DHSID', inp[4]]], on='DHSID', how='left')
print('finished calc split_label_df')
cluster_df7


 split: out of country all 2012plus size=70756, mean=23.41, std=3.25, min=15.76, max=48.11 (Whole ds)
split: out of country all 2012plus size=11358, mean=23.83, std=3.25, min=16.73, max=35.49 (Whole ds without excluded areas)
split: out of country all 2012plus                       NAN: size=59398, mean=23.17, std=3.22, min=15.76, max=48.11


split: out of country all 2012plus split 0: size=5608, mean=24.68, std=3.76, min=16.73, max=35.49
split: out of country all 2012plus split 3: size=1146, mean=23.08, std=2.69, min=17.96, max=33.85
split: out of country all 2012plus split 5: size=1137, mean=22.83, std=2.04, min=17.60, max=29.52
split: out of country all 2012plus split 2: size=1110, mean=23.18, std=2.11, min=18.01, max=31.95
split: out of country all 2012plus split 1: size=1232, mean=22.75, std=2.59, min=17.36, max=32.97
split: out of country all 2012plus split 4: size=1125, mean=23.19, std=2.29, min=17.34, max=32.40
split: out of country all 2012plus std of means: 0.65, std of stds: 0.58, std of sizes 1661.86

 split: out of country year all 2012plus size=70756, mean=23.41, std=3.25, min=15.76, max=48.11 (Whole ds)
split: out of country year all 2012plus size=11358, mean=23.83, std=3.25, min=16.73, max=35.49 (Whole ds without excluded areas)
split: out of country year all 2012plus                       NAN: size=59398, m

/home/sven/pycharm/FoodSecurity/DHS/dhs_preprocessing_functions.py:2749: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[split_col_n].iloc[int(len(df)/split_amount) * (i): int(len(df)/split_amount) * (i+1)] = 'split ' + str(i)


dropping EGGE61FL amount: 1817 mean: 29.84280507535194 vs 23.868426776516937


/home/sven/pycharm/FoodSecurity/DHS/dhs_preprocessing_functions.py:2749: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[split_col_n].iloc[int(len(df)/split_amount) * (i): int(len(df)/split_amount) * (i+1)] = 'split ' + str(i)


finished calc split_label_df


,country,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,...,split: out of country all 2012plus,split: out of country year all 2012plus,split: out of country all 2012plus excluded outlier surveys,split: out of country year all 2012plus excluded outlier surveys,split: out of adm2 all 2012plus,split: out of adm2 year all 2012plus,split: out of adm2 all 2012plus excluded outlier surveys,split: out of adm2 year all 2012plus excluded outlier surveys,split: random all 2012plus,split: random all 2012plus excluded outlier surveys
0,Angola,AO200600000001,AO,2006,1,AO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,AO200600000002,AO,2006,2,AO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AO200600000003,AO,2006,3,AO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Angola,AO200600000004,AO,2006,4,AO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AO200600000005,AO,2006,5,AO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70751,Zimbabwe,ZW201500000396,ZW,2015,396,ZW,NaN,NaN,NaN,NaN,...,split 5,split 1,split 3,split 3,split 5,split 5,split 4,split 4,split 2,split 1
70752,Zimbabwe,ZW201500000397,ZW,2015,397,ZW,NaN,NaN,NaN,NaN,...,split 5,split 1,split 3,split 3,split 1,split 0,split 1,split 5,split 1,split 1
70753,Zimbabwe,ZW201500000398,ZW,2015,398,ZW,NaN,NaN,NaN,NaN,...,split 5,split 1,split 3,split 3,split 0,split 0,split 3,split 1,split 4,split 4
70754,Zimbabwe,ZW201500000399,ZW,2015,399,ZW,NaN,NaN,NaN,NaN,...,split 5,split 1,split 3,split 3,split 5,split 5,split 4,split 4,split 3,split 4


In [10]:
for d in inputs_split_label_df:
    visualize_splits(cluster_df7, d[2], d[4], input_dir)
    stats_d = stats_of_splits(cluster_df7, d[2], d[4], d[3], stats_d)
plt.close('all')
cluster_df7.to_csv(out_f, index=False)


 split: random all 2012plus size=70756, mean=23.41, std=3.25, min=15.76, max=48.11 (Whole ds)
split: random all 2012plus size=11358, mean=23.83, std=3.25, min=16.73, max=35.49 (Whole ds without excluded areas)
split: random all 2012plus                       NAN: size=59398, mean=23.17, std=3.22, min=15.76, max=48.11
split: random all 2012plus split 5: size=1893, mean=23.80, std=3.23, min=17.36, max=35.45
split: random all 2012plus split 4: size=1893, mean=23.68, std=3.18, min=16.73, max=35.49
split: random all 2012plus split 3: size=1893, mean=23.97, std=3.29, min=17.66, max=34.89
split: random all 2012plus split 2: size=1893, mean=23.85, std=3.23, min=17.34, max=35.18
split: random all 2012plus split 0: size=1893, mean=23.93, std=3.37, min=17.36, max=35.21
split: random all 2012plus split 1: size=1893, mean=23.76, std=3.17, min=17.09, max=35.24
split: random all 2012plus std of means: 0.10, std of stds: 0.07, std of sizes 0.00

 split: random all 2012plus excluded outlier surveys si

split: random all 2012plus excluded outlier surveys size=9846, mean=22.93, std=2.29, min=16.73, max=31.11 (Whole ds without excluded areas)
split: random all 2012plus excluded outlier surveys                       NAN: size=60910, mean=23.65, std=3.59, min=15.76, max=48.11
split: random all 2012plus excluded outlier surveys split 5: size=1641, mean=22.99, std=2.27, min=17.52, max=30.46
split: random all 2012plus excluded outlier surveys split 4: size=1641, mean=22.89, std=2.28, min=17.46, max=30.83
split: random all 2012plus excluded outlier surveys split 2: size=1641, mean=22.90, std=2.26, min=17.41, max=30.71
split: random all 2012plus excluded outlier surveys split 0: size=1641, mean=22.99, std=2.33, min=17.58, max=31.07
split: random all 2012plus excluded outlier surveys split 1: size=1641, mean=22.95, std=2.34, min=17.56, max=30.68
split: random all 2012plus excluded outlier surveys split 3: size=1641, mean=22.84, std=2.25, min=16.73, max=31.11
split: random all 2012plus excluded 

In [11]:
cluster_df7['split: random all 2012plus'].dropna()

3779     split 5
3780     split 4
3781     split 5
3782     split 4
3783     split 5
          ...   
70751    split 2
70752    split 1
70753    split 4
70754    split 3
70755    split 1
Name: split: random all 2012plus, Length: 11358, dtype: object